## 1. Подготовка к работе

In [36]:
import re
from collections import defaultdict
import nltk
import pandas as pd
from torch.utils.data import Dataset
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import Trainer, TrainingArguments


In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mdchumakov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

##  2. Загрузка и предварительная обработка данных

### 2.1. Загрузка данных

In [35]:
df = pd.read_csv('data/work_data.csv')
df.head()

,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградская / ул. Печатников",Московский квартал,3,Жилой комплекс,"Московский квартал 2. Шумно : летом по ночам дикие гонки. Грязно : кругом стройки, невозможно открыть окна (16 этаж! ), вечно по району летает мусор. Детские площадки убогие, на большой площади однотипные конструкции. Очень дорогая коммуналка. Часто срабатывает пожарная сигнализация. Жильцы уже не реагируют. В это время, обычно около часа, не работают лифты. Из плюсов - отличная планировка квартир ( Московская 194 ), на мой взгляд. Ремонт от застройщика на 3-. Окна вообще жуть - вместо вентиляции. По соотношению цена/качество - 3."
1,"Московская область, Электросталь, проспект Ленина, 29",Продукты Ермолино,5,"Магазин продуктов;Продукты глубокой заморозки;Магазин мяса, колбас","Замечательная сеть магазинов в общем, хороший ассортимент, цены приемлемые, а главное качество на высоте!!! Спасибо тем, кто открыл сеть этих магазинчиков!!!!"
2,"Краснодар, Прикубанский внутригородской округ, микрорайон имени Петра Метальникова, улица Петра Метальникова, 26",LimeFit,1,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я была удивлена: 1. Хочешь что бы твой шкаф замыкался - купи замочек 2. Ты должен предоставить свой отпечаток пальца (полнейшая дичь) 3. Ставят подпись на договоре с клиентом по доверенности , графу с номером доверенности оставляют пустой , а на вопрос о номере доверенности говорят номер «2» Вы серьезно? Номер 2? Предоставить доверенность не могут, но говорят что у них в клубе «свои» доверенности, типа особенные какие-то Цирк."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. Дружелюбный персонал. Но иногда бывают неутоюбные ботинки и крепления для сноуборда . Но у меня редкий размер ноги
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5,"Салон красоты;Визажисты, стилисты;Салон бровей и ресниц","Топ мастер Ангелина топ во всех смыслах ) Немного волновалась перед посещением, потому что первый раз делала брови и ресницы. Ушла довольная максимально. Понравилось всё, от итога работы до отношения мастера. Всё объяснили рассказали, как проходит процедура, как ухаживать за результатом, даже щеточки дали для бровей))) Жаль уезжаю из Твери, так бы ходила только сюда! )"


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   address  15000 non-null  object
 1   name_ru  15000 non-null  object
 2   rating   15000 non-null  int64 
 3   rubrics  15000 non-null  object
 4   text     15000 non-null  object
dtypes: int64(1), object(4)
memory usage: 586.1+ KB


### 2.2. Фильтрация и очистка данных

Возьмем данные, которые не содержат короткие отзывы (меньше 10 символов) и не содержат пропусков.

In [39]:
def clean_review_text(text: str) -> str:
    text = text.lower()  # Приводим к нижнему регистру
    text = re.sub(r"<[^>]+>", "", text)  # Удаляем HTML-теги
    text = re.sub(r"[^\w\s,.!?()]+", "", text)  # Удаляем спецсимволы, кроме пунктуации и скобок
    text = re.sub(r"\)\)+", " ", text)  # Заменяем смайлики вида "))" на пробел
    text = re.sub(r"\s+", " ", text).strip()  # Убираем лишние пробелы
    text = re.sub(r"[\n\r]+", " ", text)  # Заменяем переносы строк на пробелы
    return text


def clean_rubrics(rubrics_: str) -> str:
    return rubrics_.lower().strip()


def clean_name_ru(name_ru: str) -> str:
    return name_ru.lower().strip()


def clean_address(address: str) -> str:
    address = address.strip()
    return re.sub(r"\s+", " ", address)


df['text'] = df['text'].apply(clean_review_text)
df['rubrics'] = df['rubrics'].apply(clean_rubrics)
df['name_ru'] = df['name_ru'].apply(clean_name_ru)
df['address'] = df['address'].apply(clean_address)

# Преобразование рейтинга в числовой формат
df['rating'] = df['rating'].astype(float)

# Удаление строк с пропущенными значениями
df.dropna(subset=['address', 'name_ru', 'rubrics', 'rating', 'text'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   address  15000 non-null  object 
 1   name_ru  15000 non-null  object 
 2   rating   15000 non-null  float64
 3   rubrics  15000 non-null  object 
 4   text     15000 non-null  object 
dtypes: float64(1), object(4)
memory usage: 586.1+ KB


In [38]:
df.head()

,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградская / ул. Печатников",московский квартал,3,жилой комплекс,"московский квартал 2. шумно летом по ночам дикие гонки. грязно кругом стройки, невозможно открыть окна (16 этаж! ), вечно по району летает мусор. детские площадки убогие, на большой площади однотипные конструкции. очень дорогая коммуналка. часто срабатывает пожарная сигнализация. жильцы уже не реагируют. в это время, обычно около часа, не работают лифты. из плюсов отличная планировка квартир ( московская 194 ), на мой взгляд. ремонт от застройщика на 3. окна вообще жуть вместо вентиляции. по соотношению ценакачество 3."
1,"Московская область, Электросталь, проспект Ленина, 29",продукты ермолино,5,"магазин продуктов;продукты глубокой заморозки;магазин мяса, колбас","замечательная сеть магазинов в общем, хороший ассортимент, цены приемлемые, а главное качество на высоте!!! спасибо тем, кто открыл сеть этих магазинчиков!!!!"
2,"Краснодар, Прикубанский внутригородской округ, микрорайон имени Петра Метальникова, улица Петра Метальникова, 26",limefit,1,фитнес-клуб,"не знаю смутят ли когото данные правила, но я была удивлена 1. хочешь что бы твой шкаф замыкался купи замочек 2. ты должен предоставить свой отпечаток пальца (полнейшая дичь) 3. ставят подпись на договоре с клиентом по доверенности , графу с номером доверенности оставляют пустой , а на вопрос о номере доверенности говорят номер 2 вы серьезно? номер 2? предоставить доверенность не могут, но говорят что у них в клубе свои доверенности, типа особенные какието цирк."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",snow-express,4,пункт проката;прокат велосипедов;сапсёрфинг,хорошие условия аренды. дружелюбный персонал. но иногда бывают неутоюбные ботинки и крепления для сноуборда . но у меня редкий размер ноги
4,"Тверь, Волоколамский проспект, 39",студия beauty brow,5,"салон красоты;визажисты, стилисты;салон бровей и ресниц","топ мастер ангелина топ во всех смыслах ) немного волновалась перед посещением, потому что первый раз делала брови и ресницы. ушла довольная максимально. понравилось всё, от итога работы до отношения мастера. всё объяснили рассказали, как проходит процедура, как ухаживать за результатом, даже щеточки дали для бровей жаль уезжаю из твери, так бы ходила только сюда! )"


### 2.3. Создание входного запроса для модели

Мы будем формировать входной текст для модели, который будет содержать категорию, рейтинг и ключевые слова. В качестве ключевых слов будем использовать рубрики.

In [40]:
PROMT_PATTERN = """
Категория: {categories}
Рейтинг: {rating}
Ключевые слова: {keywords}
Отзыв:
"""

def preprocess_row(row):
    categories = row['rubrics'].split(';')  # Предполагается, что рубрики разделены запятыми
    categories = [cat.strip() for cat in categories]
    categories_str = ', '.join(categories)
    keywords = " ".join(categories)  # Можно дополнительно извлечь ключевые слова из текста
    formatted_text = PROMT_PATTERN.format(categories=categories_str, rating=row['rating'], keywords=keywords)
    return formatted_text

df['input_text'] = df.apply(preprocess_row, axis=1)
df['target_text'] = df['text'].str.strip()

df.head()

,address,name_ru,rating,rubrics,text,input_text,target_text
0,"Екатеринбург, ул. Московская / ул. Волгоградская / ул. Печатников",московский квартал,3.0,жилой комплекс,"московский квартал 2. шумно летом по ночам дикие гонки. грязно кругом стройки, невозможно открыть окна (16 этаж! ), вечно по району летает мусор. детские площадки убогие, на большой площади однотипные конструкции. очень дорогая коммуналка. часто срабатывает пожарная сигнализация. жильцы уже не реагируют. в это время, обычно около часа, не работают лифты. из плюсов отличная планировка квартир ( московская 194 ), на мой взгляд. ремонт от застройщика на 3. окна вообще жуть вместо вентиляции. по соотношению ценакачество 3.",\nКатегория: жилой комплекс\nРейтинг: 3.0\nКлючевые слова: жилой комплекс\nОтзыв:\n,"московский квартал 2. шумно летом по ночам дикие гонки. грязно кругом стройки, невозможно открыть окна (16 этаж! ), вечно по району летает мусор. детские площадки убогие, на большой площади однотипные конструкции. очень дорогая коммуналка. часто срабатывает пожарная сигнализация. жильцы уже не реагируют. в это время, обычно около часа, не работают лифты. из плюсов отличная планировка квартир ( московская 194 ), на мой взгляд. ремонт от застройщика на 3. окна вообще жуть вместо вентиляции. по соотношению ценакачество 3."
1,"Московская область, Электросталь, проспект Ленина, 29",продукты ермолино,5.0,"магазин продуктов;продукты глубокой заморозки;магазин мяса, колбас","замечательная сеть магазинов в общем, хороший ассортимент, цены приемлемые, а главное качество на высоте!!! спасибо тем, кто открыл сеть этих магазинчиков!!!!","\nКатегория: магазин продуктов, продукты глубокой заморозки, магазин мяса, колбас\nРейтинг: 5.0\nКлючевые слова: магазин продуктов продукты глубокой заморозки магазин мяса, колбас\nОтзыв:\n","замечательная сеть магазинов в общем, хороший ассортимент, цены приемлемые, а главное качество на высоте!!! спасибо тем, кто открыл сеть этих магазинчиков!!!!"
2,"Краснодар, Прикубанский внутригородской округ, микрорайон имени Петра Метальникова, улица Петра Метальникова, 26",limefit,1.0,фитнес-клуб,"не знаю смутят ли когото данные правила, но я была удивлена 1. хочешь что бы твой шкаф замыкался купи замочек 2. ты должен предоставить свой отпечаток пальца (полнейшая дичь) 3. ставят подпись на договоре с клиентом по доверенности , графу с номером доверенности оставляют пустой , а на вопрос о номере доверенности говорят номер 2 вы серьезно? номер 2? предоставить доверенность не могут, но говорят что у них в клубе свои доверенности, типа особенные какието цирк.",\nКатегория: фитнес-клуб\nРейтинг: 1.0\nКлючевые слова: фитнес-клуб\nОтзыв:\n,"не знаю смутят ли когото данные правила, но я была удивлена 1. хочешь что бы твой шкаф замыкался купи замочек 2. ты должен предоставить свой отпечаток пальца (полнейшая дичь) 3. ставят подпись на договоре с клиентом по доверенности , графу с номером доверенности оставляют пустой , а на вопрос о номере доверенности говорят номер 2 вы серьезно? номер 2? предоставить доверенность не могут, но говорят что у них в клубе свои доверенности, типа особенные какието цирк."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",snow-express,4.0,пункт проката;прокат велосипедов;сапсёрфинг,хорошие условия аренды. дружелюбный персонал. но иногда бывают неутоюбные ботинки и крепления для сноуборда . но у меня редкий размер ноги,"\nКатегория: пункт проката, прокат велосипедов, сапсёрфинг\nРейтинг: 4.0\nКлючевые слова: пункт проката прокат велосипедов сапсёрфинг\nОтзыв:\n",хорошие условия аренды. дружелюбный персонал. но иногда бывают неутоюбные ботинки и крепления для сноуборда . но у меня редкий размер ноги
4,"Тверь, Волоколамский проспект, 39",студия beauty brow,5.0,"салон красоты;визажисты, стилисты;салон бровей и ресниц","топ мастер ангелина топ во всех смыслах ) немного волновалась перед посещением, потому что первый раз делала брови и ресницы. ушла довольная максимально. понравилось всё, от итога работы до отношения мастера. всё объ

### 2.5. Разделение данных на обучающую и тестовую выборки


In [41]:
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

print(f"Размер обучающей выборки: {len(train_df)}")
print(f"Размер валидационной выборки: {len(val_df)}")

Размер обучающей выборки: 13500
Размер валидационной выборки: 1500


## 3. Создание датасета и загрузчика данных

Используем PyTorch для создания пользовательского датасета.

In [42]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer

class ReviewsDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer, max_length=512):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_enc = self.tokenizer.encode_plus(
            self.inputs[idx],
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        target_enc = self.tokenizer.encode_plus(
            self.targets[idx],
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        labels = target_enc['input_ids']
        labels[labels == self.tokenizer.pad_token_id] = -100  # Игнорирование паддинга при расчете лосса

        return {
            'input_ids': input_enc['input_ids'].squeeze(),
            'attention_mask': input_enc['attention_mask'].squeeze(),
            'labels': labels.squeeze()
        }

# Выбор модели и токенизатора
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Добавление специальных токенов, если необходимо
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Например, добавление PAD токена

# Создание датасетов
train_dataset = ReviewsDataset(
    inputs=train_df['input_text'].tolist(),
    targets=train_df['target_text'].tolist(),
    tokenizer=tokenizer
)

val_dataset = ReviewsDataset(
    inputs=val_df['input_text'].tolist(),
    targets=val_df['target_text'].tolist(),
    tokenizer=tokenizer
)

# Создание загрузчиков данных
batch_size = 8

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)



## 4. Выбор и настройка модели

Мы будем использовать предобученную модель ruGPT-3 Small, предоставленную SberAI. Вы можете выбрать другую модель из Hugging Face Model Hub (https://huggingface.co/models).

In [43]:
# Загрузка предобученной модели
model = GPT2LMHeadModel.from_pretrained(model_name)

## 5. Обучение модели

Используем Trainer из Hugging Face для упрощения процесса обучения.

In [44]:
from transformers import AdamW, get_linear_schedule_with_warmup
import torch.nn as nn

# Проверка наличия GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Параметры обучения
epochs = 3
learning_rate = 5e-5
warmup_steps = 500

# Определение оптимизатора и планировщика
optimizer = AdamW(model.parameters(), lr=learning_rate)

total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

# Критерий потерь
criterion = nn.CrossEntropyLoss(ignore_index=-100)

# Функция обучения на одной эпохе
def train_epoch(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc="Training"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

# Функция валидации на одной эпохе
def eval_epoch(model, dataloader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Validation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

from tqdm import tqdm

# Цикл обучения
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")

    train_loss = train_epoch(model, train_loader, optimizer, scheduler, device)
    print(f"Training loss: {train_loss:.4f}")

    val_loss = eval_epoch(model, val_loader, device)

    print(f"Validation loss: {val_loss:.4f}")

    # Можно добавить сохранение модели после каждой эпохи
    model.save_pretrained(f'model_epoch_{epoch + 1}')
    tokenizer.save_pretrained(f'model_epoch_{epoch + 1}')


# После завершения обучения сохраним модель и токенизатор для дальнейшего использования.
model_save_path = 'models/model_v2'
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)


/Users/mdchumakov/miniconda3/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



Epoch 1/3


Training: 100%|██████████| 1688/1688 [3:24:40<00:00,  7.28s/it]  


Training loss: 7.7617


Validation: 100%|██████████| 188/188 [03:25<00:00,  1.09s/it]


...

Training: 100%|██████████| 1688/1688 [3:25:03<00:00,  7.29s/it]  


Training loss: 7.5703


Validation: 100%|██████████| 188/188 [03:30<00:00,  1.12s/it]


Validation loss: 7.5814


('models/model_v2/tokenizer_config.json',
 'models/model_v2/special_tokens_map.json',
 'models/model_v2/vocab.json',
 'models/model_v2/merges.txt',
 'models/model_v2/added_tokens.json')

## 6. Генерация отзывов

После обучения модели можно использовать её для генерации новых отзывов на основе заданных параметров.

In [68]:
def generate_review(category, rating, keywords, max_length=150, temperature=0.7, top_k=50, top_p=0.95):
    prompt = f"Категория: {category}\nРейтинг: {rating}\nКлючевые слова: {keywords}\nОтзыв:"
    inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)

    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=inputs.shape[1] + max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            do_sample=True,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Извлечение только текста отзыва
    review = generated_text.split('Отзыв:')[-1].strip()
    return review


category = "ресторан"
rating = 5
keywords = "паста"

generated_review = generate_review(category, rating, keywords, 15, 1, 100000, 1.5)
print("Сгенерированный отзыв:")
print(generated_review)

Сгенерированный отзыв:
ные вола что на для внет вот замечаетли поблагодарить в оригинал рекомендуем


In [22]:
rubrics='Рестораны',
rating=4,
keywords='вкусно, обслуживание, уют'
prompt = f"Категория: {rubrics}\nРейтинг: {rating}/5\nКлючевые слова: {keywords}\nОтзыв:"
input_ids = tokenizer.encode(prompt, return_tensors='pt')
model.generate(input_ids)
# model

tensor([[ 6207,  4591,  1183,    30,   473,    11,  8706, 26322,   318, 22493,
            13,   203, 41342,  7851,    30,   473,    24,    16,    13,    19,
            25,   203,   684,   623,   390,  1936,  1694,    30, 24636,    16,
         16495,    16, 10360,   203, 50257,    16,   369,    16,   369,    16,
           369,    16,   369,    16,   369,    16,   369,    16,   369,    16,
           369,    16,   369,    16,   369]])

In [34]:
model.save_pretrained("models/model_v1")
tokenizer.save_pretrained("models/model_v1")

('models/model_v1/tokenizer_config.json',
 'models/model_v1/special_tokens_map.json',
 'models/model_v1/vocab.json',
 'models/model_v1/merges.txt',
 'models/model_v1/added_tokens.json')